In [7]:
import ebaysdk
from ebaysdk.finding import Connection as finding

from bs4 import BeautifulSoup
import requests
import re 
import pandas as pd


keywords = list(pd.read_fwf("keywords_CamerasPhoto.txt", delim_whitespace = True).ix[:,1])
api = finding(siteid='EBAY-US', appid='MatiasIg-matuteig-PRD-c5d7504c4-8bbb8189')


In [8]:
results = []
entriesPerPage = 100
for k in keywords:
    print k
    api.execute('findItemsAdvanced', {
        'keywords': k,
        'categoryId' : ['625'],
        'itemFilter': [
            {'name': 'MinPrice', 'value': '30', 'paramName': 'Currency', 'paramValue': 'EUR'},
            {'name': 'MaxPrice', 'value': '100', 'paramName': 'Currency', 'paramValue': 'EUR'},
        ],
        'descriptionSearch': '1',
        'paginationInput': {
            'entriesPerPage': str(entriesPerPage),
            'pageNumber': '1' 	 
        },
        'sortOrder': 'CurrentPriceHighest'
    })
    results = results + [api.response.dict()]
    


hasselblad
gopro
minolta
len
polaroid
rolleiflex
tripod
ony
telescope
gopro hero 3
canon digital camera
video camera
graflex
canon len
voigtlander
mamiya
nikon len
manfrotto
camcorder
alpa
linhof
angenieux
polaroid camera
inar
olympu
norman
phase one
gopro hero 3 black edition
camera bag
brass len
d card
zeiss ikon
binocular
contax
ony nex
arri
gopro hero 3 black
vintage camera
miranda
really right stuff
zei
profoto
digital back
dslr
takumar
light stand
cambo
reflector
dallmeyer
nikkor
horseman
olympus om
contax 645
oftbox
cooke
contarex
alien bee
canon eo
wollensak
exakta
teadicam
red one
rollei 35
mamiya 645
large format camera
pelican case
nikon coolpix
deardorff
light meter
chneider
flash
gopro hero 2
pentax 67
nikon rangefinder
dynalite
anamorphic
petri
canon rangefinder
rolleicord
memory card
praktica
rangefinder
yashica
arriflex
pentacon
pocket wizard
gitzo
matthew
elinchrom
canon eos m
mamiya 7
achtler
red one camera
canon rebel
quantum
bronica
nikon camera
nikon f
large format

In [9]:
results_df = pd.DataFrame(columns=['categoryId', 'categoryName', '_currencyId', 'value', 'title', 'viewItemURL'])
j = 0

for r in results:
    try:
        for i in range(len(r['searchResult']['item'])):
            details = r['searchResult']['item'][i]
            try:
                categoryId = details['primaryCategory']['categoryId']
            except KeyError:
                categoryId = None
            try:
                categoryName =  details['primaryCategory']['categoryName']
            except KeyError:
                categoryName = None
            try:
                _currencyId = details['sellingStatus']['currentPrice']['_currencyId']
            except KeyError:
                _currencyId = None
            try:
                value = details['sellingStatus']['currentPrice']['value']
            except KeyError:
                value = None
            try:
                title = details['title']
            except KeyError:
                title = None
            try:
                viewItemURL = details['viewItemURL']
            except KeyError:
                viewItemURL = None

            results_df.loc[j] = [categoryId, categoryName, _currencyId, value, title, viewItemURL]
            j+=1
    except KeyError:
        pass
    
results_df.to_csv('results_df.csv', index = None, encoding = 'utf-8')

In [10]:
import pandas as pd
from time import sleep

urls = results_df['viewItemURL'].unique()

sales_df = pd.DataFrame(columns=['title', 'viewItemURL', 'price', 'sales'])
i = 0
# for r in range(len(results)):
#     print 'SEARCH WITH KEYWORDS: '+str(keywords[r])
for url in urls:
    i +=1
    try:
        r = requests.get(url)
    except requests.exceptions.ConnectionError:
        print "Connection refused event"
        sleep(5)
        r = requests.get(url)

    x = BeautifulSoup(r.content, "html.parser")
    title = x.find_all('title')[0].string
    try:
        price = 0.01*int(filter(unicode.isdigit, x.find_all('span', itemprop="price")[0].string.strip(u'\xa3\x20AC\x24')))
    except IndexError:
        try:
            price = 0.01*int(filter(unicode.isdigit, x.find_all('span', id="mm-saleDscPrc")[0].string.strip(u'\xa3\x20AC\x24')))
        except IndexError:
            try:
                price = 0.01*int(filter(unicode.isdigit, x.find_all('span', id="prcIsum")[0].string.strip(u'\xa3\x20AC\x24')))
            except IndexError:
                price = 0.01*int(filter(unicode.isdigit, x.find_all('span', class_='notranslate vi-VR-cvipPrice')[0].string.strip(u'\xa3\x20AC\x24')))
    try:
        sales = int(filter(unicode.isdigit, x.find_all('a', href=lambda x: x and 'ViewBidsLogin' in x)[0].string))
    except IndexError:
        sales = 0

    sales_df.loc[i] = [title, url, price, sales]
    if i%20 == 0:
        sales_df.to_csv('ScrapedData2.csv', index = None, encoding = 'utf-8')
        

Connection refused event
Connection refused event


KeyboardInterrupt: 

In [40]:
# print x.prettify()
sales_df = pd.read_csv('ScrapedData2.csv').drop_duplicates('url')
results_df = pd.read_csv('results_df.csv').drop_duplicates('viewItemURL')


In [41]:
sales_df['viewItemURL'] = sales_df['url'] 

In [45]:
data = sales_df.merge(results_df, on = ['viewItemURL'])[['title_x','categoryId','categoryName', 'price','_currencyId', 'value', 'sales']]
data.loc[data['sales']>5].sort_values(by = '_currencyId')

,title_x,categoryId,categoryName,price,_currencyId,value,sales
806,TurnsPro - 360° Timelapse Panning device for G...,27432,"Other Cameras, Accessories",149.50,AUD,149.50,93.0
39,Dimmable 14&apos;&apos; 40W LED 5500K Dimmable...,3860,Other Continuous Lighting,115.00,USD,115.00,37.0
617,FOTGA DP3000 15mm Rod Baseplate+Shoulder Pad+H...,3319,Other Production & Editing,116.10,USD,116.10,41.0
637,FEELWORLD 7 Inch TFT LCD HD 1080P HDMI DSLR Ca...,21517,Video Monitors,110.99,USD,110.99,215.0
744,Digital Infrared Night Vision 5X42 Monocular H...,83880,Binoculars & Monoculars,116.01,USD,116.01,9.0
751,1.25 Inch 9 Piece Plossl Telescope Eyepiece Se...,29954,Telescope Eyepieces & Lenses,115.99,USD,115.99,14.0
752,ZOMEI Professional Carbon Fiber Travel Ball He...,30093,Tripods & Monopods,115.99,USD,115.99,8.0
753,Nikon 18-55mm f/3.5-5.6G VR Lens + Accessories...,3323,Lenses,115.99,USD,115.99,27.0
755,688C Professional Portable Carbon Fiber Tripod...,30093,Tripods & Monopods,115.99,USD,115.99,148.0
775,"1.25"" New 9 Piece Plossl Multicoated Eyepiece ...",29954,Telescope Eyepieces & Lenses,115.89,USD,115.89,8.0
